**MASK WEAR DETECTION WITH LOGISTIC REGRESSION :**

In [1]:
from torch.utils.data import DataLoader
import os
import cv2
import random

class DataSet :

    def __init__(self,root,labels):
        """Init function should not do any heavy lifting, but
            must initialize how many items are available in this data set.
        """

        self.ROOT = root
        self.images = read_images(root)
        self.labels = torch.from_numpy(np.array(labels))
        #self.labels = read_labels(labels)

    def __len__(self):
        """return number of elements in our dataset"""
        
        return len(self.images)

    def __getitem__(self, idx):
        """ Here we have to return the item requested by `idx`
            The PyTorch DataLoader class will use this method to make an iterable for
            our training or validation loop.
        """

        img = self.images[idx]
        label = self.labels[idx]

        return img, label
    
    

                    
            
            
        
        

In [12]:
import torch
import numpy as np

def read_images(root) :
        '''load all the images from a folder'''
        images = []
        for filename in os.listdir(root) :
            '''
            Python method listdir() returns a list containing the names of the 
            entries in the directory given by path. The list is in arbitrary order. 
            It does not include the special entries '.' and '..' even if they are 
            present in the directory.'''
            img = cv2.imread(os.path.join(root,filename))
            if img is not None:
                img = img[:,:,::-1]/255 # To shuffle the color channels form BGR to RGB and /255 to normalize
                # A REGARDER COMMENT CA MARCHE PLUS EN DETAIL 
                images.append(img)
        return torch.from_numpy(np.array(images))
    



ds_uwm = DataSet('U',[0 for k in range(20)])
ds_wm = DataSet("W",[1 for k in range(20)])

Let's display the size of the datasets :

In [13]:
print("Worn Mask Dataset Length : {}\n".format(len(ds_wm)))
print("Unworn Mask Dataset Length : {}".format(len(ds_uwm)))

Worn Mask Dataset Length : 30

Unworn Mask Dataset Length : 30


Let's show the 6th element of the unworn mask dataset :

In [14]:
img, label = ds_uwm[5]
print(img,label)
print("\nType of the image : {}\n".format(type(img)))
print("Shape of the image : {}".format(img.shape))

tensor([[[0.1451, 0.0980, 0.1020],
         [0.1412, 0.1020, 0.0980],
         [0.1490, 0.1098, 0.1059],
         ...,
         [0.5333, 0.5373, 0.6745],
         [0.5294, 0.5412, 0.6706],
         [0.5294, 0.5451, 0.6667]],

        [[0.1608, 0.1137, 0.1020],
         [0.1608, 0.1098, 0.1059],
         [0.1686, 0.1216, 0.1176],
         ...,
         [0.5333, 0.5490, 0.6824],
         [0.5294, 0.5490, 0.6745],
         [0.5255, 0.5451, 0.6667]],

        [[0.1529, 0.1098, 0.0902],
         [0.1686, 0.1137, 0.1098],
         [0.1804, 0.1255, 0.1216],
         ...,
         [0.5451, 0.5608, 0.6824],
         [0.5412, 0.5569, 0.6824],
         [0.5373, 0.5569, 0.6745]],

        ...,

        [[1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000],
         ...,
         [0.2980, 0.2784, 0.1725],
         [0.2980, 0.2824, 0.1686],
         [0.2863, 0.2706, 0.1529]],

        [[1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000],
         [1.

# Training and Validation Datasets

While building real world machine learning models, it is quite common to split the dataset into 3 parts:

1. **Training set** - used to train the model i.e. compute the loss and adjust the weights of the model using gradient descent.
2. **Validation set** - used to evaluate the model while training, adjust hyperparameters (learning rate etc.) and pick the best version of the model.
3. **Testing set** - used to compare different models, or different types of modeling approaches, and report the final accuracy of the model.

In [18]:
from torch.utils.data import random_split
from torch.utils.data import ConcatDataset

train_ds_wm, temp = random_split(ds_wm, [10, 20])
val_ds_wm, test_ds_wm = random_split(temp, [10, 10])

train_ds_uwm, temp = random_split(ds_uwm, [10, 20])
val_ds_uwm, test_ds_uwm = random_split(temp, [10, 10])

train_ds = train_ds_uwm + train_ds_wm
val_ds = val_ds_uwm + val_ds_wm
test_ds = test_ds_uwm + test_ds_wm

print("Length of the training set : {}\n".format(len(train_ds)))
print("Length of the validation set = {}\n".format(len(val_ds)))
print("Length of the testing set = {}".format(len(test_ds)))

Length of the training set : 20

Length of the validation set = 20

Length of the testing set = 20


Let's initialize the dataloader :

In [20]:
from torch.utils.data import DataLoader

# The most important argument of DataLoader constructor is dataset, which indicates a dataset object to load data from. 
# PyTorch supports two different types of datasets:
#        - map-style datasets,
#        - iterable-style datasets"""

batch_size = 10

train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

# We set shuffle=True for the training dataloader, so that the batches generated in each epoch are different, and this 
# randomization helps generalize & speed up the training process. 
# On the other hand, since the validation dataloader is used only for evaluating the model, there is no need to shuffle 
# the images.

Let's initialize the model :

In [22]:
import torch.nn as nn

input_size = 1024*1024
num_classes = 2

# Logistic regression model
model = nn.Linear(input_size, num_classes)

In [26]:
print("Display the shape of the model weights : {}\n".format(model.weight.shape))
print("Display all the model weights : {}\n".format(model.weight))

Display the shape of the model weights : torch.Size([2, 1048576])

Display all the model weights : Parameter containing:
tensor([[ 7.5618e-04, -1.7856e-04,  4.4048e-05,  ...,  8.5169e-04,
         -2.1167e-04,  6.2061e-04],
        [-1.4814e-04, -4.8100e-04,  2.9299e-04,  ..., -8.7685e-04,
         -8.6172e-04,  3.7356e-04]], requires_grad=True)



In [28]:
print("Display the shape of the model bias : {}\n".format(model.bias.shape))
print("Display all the model bias : {}\n".format(model.bias))

Display the shape of the model bias : torch.Size([2])

Display all the model bias : Parameter containing:
tensor([-0.0010,  0.0005], requires_grad=True)



In [153]:
for images, labels in train_ds:
    outputs = model(images)
    break

tensor(0, dtype=torch.int32)
torch.Size([1024, 1024, 3])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1048576x3 and 1048576x2)

In [154]:
class MWDModel(nn.Module): # MWD for Mask Wear detection
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, num_classes)
        
    def forward(self, xb):
        xb = xb.reshape(-1, 1048576)
        out = self.linear(xb)
        return out
    
model = MWDModel()

In [155]:
print(model.linear.weight.shape, model.linear.bias.shape)
list(model.parameters())

torch.Size([2, 1048576]) torch.Size([2])


[Parameter containing:
 tensor([[ 9.3787e-04, -1.5346e-04, -3.9160e-04,  ..., -3.6419e-05,
          -1.7810e-04, -5.3560e-04],
         [ 1.2859e-04,  5.6075e-05,  5.0963e-04,  ...,  7.5156e-05,
           3.9334e-05,  4.2921e-04]], requires_grad=True),
 Parameter containing:
 tensor([-0.0002, -0.0004], requires_grad=True)]

In [156]:
for images, labels in train_ds:
    outputs = model(images.float()) # FLOAT()
    break

print('outputs.shape : ', outputs.shape)
print('Sample outputs :\n', outputs[:2].data)

# When the error is RuntimeError: Expected object of scalar type Float but got scalar type Double for argument #4 'mat1', you 
# would need to use the .float() function since it says Expected object of scalar type Float.

# Therefore, the solution is changing y_pred = model(X_trainTensor) to y_pred = model(X_trainTensor.float()).

# Likewise, when you get another error for loss = loss_fn(y_pred, y_trainTensor), you need y_trainTensor.long() since the 
# error message says Expected object of scalar type Long.

# You could also do model.double()'''

outputs.shape :  torch.Size([3, 2])
Sample outputs :
 tensor([[ 0.4541,  0.0132],
        [ 0.3792, -0.1086]])


In [157]:
import torch.nn.functional as F

In [158]:
# Apply softmax for each output row
probs = F.softmax(outputs, dim=1)

# Look at sample probabilities
print("Sample probabilities:\n", probs[:2].data)

# Add up the probabilities of an output row
print("Sum: ", torch.sum(probs[0]).item())

Sample probabilities:
 tensor([[0.6085, 0.3915],
        [0.6196, 0.3804]])
Sum:  1.0


In [159]:
max_probs, preds = torch.max(probs, dim=1)
print(preds)
print(max_probs)

tensor([0, 0, 1])
tensor([0.6085, 0.6196, 0.5232], grad_fn=<MaxBackward0>)


In [160]:
labels

tensor(0, dtype=torch.int32)

In [161]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [162]:
accuracy(outputs, labels)

tensor(0.6667)

In [163]:
loss_fn = F.cross_entropy

In [168]:
# Loss for current batch of data
loss = loss_fn(outputs, labels)
print(loss)

IndexError: dimension specified as 0 but tensor has no dimensions

In [165]:
class MWDModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, num_classes)
        
    def forward(self, xb):
        xb = xb.reshape(-1, 1048576)
        out = self.linear(xb)
        return out
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images.float())                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images.float())                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss, 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))
    
model = MWDModel()

In [166]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    print(outputs)
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [167]:
result0 = evaluate(model, val_loader)
result0

ValueError: Expected input batch_size (60) to match target batch_size (20).

In [52]:
history1 = fit(5, 0.001, model, train_loader, val_loader)

ValueError: Expected input batch_size (30) to match target batch_size (10).

In [ ]:
history2 = fit(5, 0.001, model, train_loader, val_loader)

In [65]:
history3 = fit(5, 0.001, model, train_loader, val_loader)

NameError: name 'train_loader' is not defined

In [66]:
history4 = fit(5, 0.001, model, train_loader, val_loader)

NameError: name 'train_loader' is not defined

In [ ]:
# Replace these values with your results
history = [result0] + history1 + history2 + history3 + history4
accuracies = [result['val_acc'] for result in history]
plt.plot(accuracies, '-x')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title('Accuracy vs. No. of epochs');